In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
TICKERS = ["21STCENMGM.NS", "21STCENMGM.BO"]

DAYS_LOOKBACK = 365
TOP_N = 5

In [3]:
def safe_is_nan(x):
    """Returns True if x is NaN, False otherwise."""
    try:
        return pd.isna(float(x))
    except:  # noqa: E722
        return True

In [4]:
def pct_return(close_prices, days):
    if len(close_prices) <= days:
        return np.nan
    try:
        return float((close_prices.iloc[-1] / close_prices.iloc[-days - 1] - 1) * 100)
    except Exception as e:
        print("pct_return error:", e)
        return np.nan

In [5]:
def annualized_vol(close_prices, window=30):
    try:
        log_ret = np.log(close_prices / close_prices.shift(1)).dropna()
        if len(log_ret) < window:
            return np.nan
        vol = float(log_ret.rolling(window).std().iloc[-1])
        return vol * np.sqrt(252) * 100
    except Exception as e:
        print("annualized_vol error:", e)
        return np.nan

In [6]:
def fetch_info_safe(ticker):
    try:
        return yf.Ticker(ticker).info
    except:  # noqa: E722
        return {}

In [7]:
def analyze_stocks(tickers):
    start = (datetime.today() - timedelta(days=DAYS_LOOKBACK)).strftime("%Y-%m-%d")
    end = datetime.today().strftime("%Y-%m-%d")

    results = []

    for t in tickers:
        print(f"\nFetching: {t}")

        try:
            data = yf.download(
                t,
                start=start,
                end=end,
                auto_adjust=False,
                progress=False,
                threads=False,
            )

            if data is None or data.empty:
                print(f"⚠ No price data for {t}")
                continue

            close = data["Close"]

            r_1m = pct_return(close, 30)
            r_3m = pct_return(close, 90)
            r_6m = pct_return(close, 180)
            vol30 = annualized_vol(close)

            print(" - 1m:", r_1m, "type:", type(r_1m))
            print(" - 3m:", r_3m, "type:", type(r_3m))
            print(" - 6m:", r_6m, "type:", type(r_6m))
            print(" - vol30:", vol30, "type:", type(vol30))

            # momentum safe calculation
            if (
                safe_is_nan(r_1m)
                or safe_is_nan(r_3m)
                or safe_is_nan(r_6m)
                or safe_is_nan(vol30)
            ):
                momentum = np.nan
            else:
                momentum = float(
                    (0.5 * r_6m + 0.3 * r_3m + 0.2 * r_1m) / (vol30 + 1e-6)
                )

            info = fetch_info_safe(t)
            market_cap = info.get("marketCap", np.nan)
            pe = info.get("trailingPE", np.nan)

            results.append(
                {
                    "ticker": t,
                    "last_close": float(close.iloc[-1]),
                    "1m_return%": r_1m,
                    "3m_return%": r_3m,
                    "6m_return%": r_6m,
                    "vol30_ann%": vol30,
                    "momentum": momentum,
                    "market_cap": market_cap,
                    "pe": pe,
                }
            )

        except Exception as e:
            print(f"❌ Error while processing {t}: {e}")

    return pd.DataFrame(results)

In [8]:
def compute_scores(df):
    if df.empty:
        return df

    df["mom_z"] = (df["momentum"] - df["momentum"].mean()) / (
        df["momentum"].std() + 1e-9
    )

    df["mc_log"] = np.log1p(df["market_cap"].fillna(0))
    df["mc_z"] = (df["mc_log"] - df["mc_log"].mean()) / (df["mc_log"].std() + 1e-9)

    df["score"] = 0.75 * df["mom_z"].fillna(-5) + 0.25 * df["mc_z"].fillna(-5)

    return df.sort_values("score", ascending=False)

In [ ]:
def main():
    df = analyze_stocks(TICKERS)

    if df.empty:
        print("\n❌ No stocks could be analyzed. Exiting.")
        return

    df = df.set_index("ticker")
    df_scored = compute_scores(df)

    print("\nTop Recommended Delivery-Related Stocks:\n")
    print(
        df_scored[
            [
                "last_close",
                "1m_return%",
                "3m_return%",
                "6m_return%",
                "vol30_ann%",
                "pe",
                "market_cap",
                "score",
            ]
        ]
        .head(TOP_N)
        .to_string()
    )

    df_scored.to_csv("delivery_stock_scores.csv")
    print("\n✔ Full scores saved to delivery_stock_scores.csv")

In [10]:
if __name__ == "__main__":
    main()


Fetching: 21STCENMGM.NS


C:\Users\brame\AppData\Local\Temp\ipykernel_12508\3099157628.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float((close_prices.iloc[-1] / close_prices.iloc[-days - 1] - 1) * 100)
C:\Users\brame\AppData\Local\Temp\ipykernel_12508\746945723.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  vol = float(log_ret.rolling(window).std().iloc[-1])


 - 1m: -18.83305590440445 type: <class 'float'>
 - 3m: -35.558852207521795 type: <class 'float'>
 - 6m: -44.16512075442625 type: <class 'float'>
 - vol30: 22.731213128879517 type: <class 'numpy.float64'>


C:\Users\brame\AppData\Local\Temp\ipykernel_12508\2449680940.py:56: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "last_close": float(close.iloc[-1]),



Fetching: 21STCENMGM.BO


C:\Users\brame\AppData\Local\Temp\ipykernel_12508\3099157628.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float((close_prices.iloc[-1] / close_prices.iloc[-days - 1] - 1) * 100)
C:\Users\brame\AppData\Local\Temp\ipykernel_12508\746945723.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  vol = float(log_ret.rolling(window).std().iloc[-1])


 - 1m: -18.40063639500412 type: <class 'float'>
 - 3m: -33.83870893909086 type: <class 'float'>
 - 6m: -37.4409018626218 type: <class 'float'>
 - vol30: 22.689777170977894 type: <class 'numpy.float64'>

Top Recommended Delivery-Related Stocks:

               last_close  1m_return%  3m_return%  6m_return%  vol30_ann%  pe  market_cap     score
ticker                                                                                             
21STCENMGM.BO       41.02  -18.400636  -33.838709  -37.440902   22.689777 NaN   430710016  0.707107
21STCENMGM.NS       39.09  -18.833056  -35.558852  -44.165121   22.731213 NaN   410444992 -0.707107

✔ Full scores saved to delivery_stock_scores.csv


C:\Users\brame\AppData\Local\Temp\ipykernel_12508\2449680940.py:56: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "last_close": float(close.iloc[-1]),
